# Hyperparameter Tuning

## Nested Cross-Validation
The process of tuning hyperparameters is a form of machine learning, thus, it needs to be properly cross-validated. The tuning step has *only* generated the best hyperparameters from the cross-validation it did with the data.

With nested cross-validation, an additional cross-validation needs to be performed only on models with these new hyperparameters. Therefore, the outer cross-validation (which evaluates the model *after* hyperparameters have been selected) will generate various train-test splits. Then, the inner cross-validation will generate its own train-test splits from the train dataset of the outer cross-validation.

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.svm import SVC

data, target = load_breast_cancer(return_X_y=True)
param_grid = {"C": [0.1, 1.0, 10.0], "gamma": [0.01, 0.1]}

inner_cv = KFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=0)

model = GridSearchCV(SVC(), param_grid=param_grid, cv=inner_cv, n_jobs=2)
test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=2)
test_score

array([0.64210526, 0.63157895, 0.60846561])